In [ ]:
import pandas as pd

# Pitcher
Determine best pitcher per year. Can do line graph of avg IPouts per game

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pitching = pd.read_csv('baseballdatabank-2022.2/core/Pitching.csv')
pitching = pitching[['playerID', 'yearID', 'W', 'G','IPouts', 'BFP']]
players = pd.read_csv('baseballdatabank-2022.2/core/People.csv')
players = players [['playerID', 'nameFirst', 'nameLast']]
pitchers = pd.merge(players, pitching, on='playerID')
pitchers.head()

In [ ]:
pitchers.rename(columns ={'BFP': "battersFaced"})

In [ ]:
##Finding average outs per game from pitcher for the year
pitchers['pitchAvg'] = pitchers.IPouts / pitchers.G
pitchers.head()

In [ ]:
##20 best pitchers by pitching average - how to filter outliers (low amount of games)
pitchers.sort_values(by='pitchAvg', ascending=False).head(20)

# Manager
Determine best manager. Can do lineplot for managers and wins. Determine if manager or team has more wins (don't know how to calculate if manager worked for multiple teams)

In [ ]:
teams = pd.read_csv('baseballdatabank-2022.2/core/Teams.csv')
teams = teams[['teamID','yearID','G','W','L']]
teams.head()

In [ ]:
teams['wonGamesPercent'] = teams['W'] / teams['G']
teams.sort_values(by='W', ascending=False).head(20)

In [ ]:
managers = pd.read_csv('baseballdatabank-2022.2/core/Managers.csv')
managers = managers[['playerID', 'teamID', 'yearID','G','W','L']]
managers.head()

In [ ]:
##20 best players by games won
managers['wonGamesPercent'] = managers['W'] / managers['G']
managers.sort_values(by='W', ascending=False).head(20)